# 2D Scanning
## Import libraries

In [1]:
# Base libraries
import math
import numpy as np
import scipy.integrate as integrate
from tqdm import tqdm
from scipy.special import erf
import pickle
import itertools

from SALib.sample import saltelli
from SALib.analyze import sobol

# Personal libraries
import sixtrackwrap as sx

from parameters import *

## Computing

In [2]:
data_5 = {}
DA_5 = {}
error_5 = {}

alpha_values = np.linspace(0, np.pi * 0.5, alpha_max_samples)

engine = sx.radial_scanner(alpha_values, np.zeros(alpha_values.shape), np.zeros(alpha_values.shape), d_r, starting_step=starting_step)
engine.scan(turn_sampling[0], turn_sampling[-1])
radiuses = engine.extract_DA(turn_sampling)
data_5 = radiuses

skips = [1]
while True:
    if (alpha_max_samples - 1) // skips[-1] > 4:
        skips.append(skips[-1] * 2)
    else:
        break

for i in skips:
    alpha = alpha_values[::i]
    d_alpha = alpha[1] - alpha[0]
    cutted_radiuses = radiuses[::i]
    value = integrate.simps(cutted_radiuses ** 2, alpha, axis=0)
    less_value = integrate.simps(cutted_radiuses[::2] ** 2, alpha[::2], axis=0)
    uncertainty = np.abs((value - less_value))

    DA = np.sqrt(value * 2 / np.pi)
    uncertainty = 0.5 * np.power(value * 2 / np.pi, -0.5) * uncertainty
    DA_5[cutted_radiuses.shape] = np.asarray(DA)
    error_5[cutted_radiuses.shape] = uncertainty 

Active radiuses: 513 / 513
Sample size per active radius: 4
Expected execution time for step: nan
Time per single iteration: 2.2412616490852994e-05
r: 13.0 . Turns to do: 1000 . Min found: 1000
Active radiuses: 513 / 513
Sample size per active radius: 4
Expected execution time for step: 45.99
Time per single iteration: 2.23776291453118e-05
r: 17.0 . Turns to do: 1000 . Min found: 143
Active radiuses: 513 / 513
Sample size per active radius: 4
Expected execution time for step: 45.92
Time per single iteration: 2.1957401876096373e-05
r: 21.0 . Turns to do: 1000 . Min found: 50
Active radiuses: 336 / 513
Sample size per active radius: 7
Expected execution time for step: 51.64
Time per single iteration: 1.918377225496331e-05
r: 28.0 . Turns to do: 449 . Min found: 3
Active radiuses: 26 / 513
Sample size per active radius: 79
Expected execution time for step: 17.69
Time per single iteration: 1.7668903842794765e-05
r: 107.0 . Turns to do: 0 . Min found: 0
TOTAL ELAPSED TIME IN SECONDS: 198.38

### Saving Data

In [3]:
with open("data/DA_5.pkl", 'wb') as f:
    pickle.dump(DA_5, f, protocol=4)
    
with open("data/error_5.pkl", 'wb') as f:
    pickle.dump(error_5, f, protocol=4)